In [1]:
import numpy as np
import warnings
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import numpy as np
from catboost import CatBoostClassifier, Pool

# Загрузка и предобработка данных (1)

In [2]:
data = pd.read_csv("Bank_pred.csv")
print(data.shape)
data.head()

(10000, 12)


,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df_c = data.copy()
df_c = df_c.drop(["customer_id"], axis = 1)
df_c.head()

,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df_c.isna().sum()

credit_score        0
country             0
gender              0
age                 0
tenure              0
balance             0
products_number     0
credit_card         0
active_member       0
estimated_salary    0
churn               0
dtype: int64

### Разделение на X,y

In [5]:
X = df_c.drop(["churn"], axis = 1)
y = df_c['churn']

In [6]:
X_dummies= pd.get_dummies(X, drop_first=True)

In [7]:
corr_matrix = pd.concat([X_dummies, y], axis=1).corr()
corr_matrix

,credit_score,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,country_Germany,country_Spain,gender_Male,churn
credit_score,1.000000,-0.003965,0.000842,0.006268,0.012238,-0.005458,0.025651,-0.001384,0.005538,0.004780,-0.002857,-0.027094
age,-0.003965,1.000000,-0.009997,0.028308,-0.030680,-0.011721,0.085472,-0.007201,0.046897,-0.001685,-0.027544,0.285323
tenure,0.000842,-0.009997,1.000000,-0.012254,0.013444,0.022583,-0.028362,0.007784,-0.000567,0.003868,0.014733,-0.014001
balance,0.006268,0.028308,-0.012254,1.000000,-0.304180,-0.014858,-0.010084,0.012797,0.401110,-0.134892,0.012087,0.118533
products_number,0.012238,-0.030680,0.013444,-0.304180,1.000000,0.003183,0.009612,0.014204,-0.010419,0.009039,-0.021859,-0.047820
credit_card,-0.005458,-0.011721,0.022583,-0.014858,0.003183,1.000000,-0.011866,-0.009933,0.010577,-0.013480,0.005766,-0.007138
active_member,0.025651,0.085472,-0.028362,-0.010084,0.009612,-0.011866,1.000000,-0.011421,-0.020486,0.016732,0.022544,-0.156128
estimated_salary,-0.001384,-0.007201,0.007784,0.012797,0.014204,-0.009933,-0.011421,1.000000,0.010297,-0.006482,-0.008112,0.012097
country_Germany,0.005538,0.046897,-0.000567,0.401110,-0.010419,0.010577,-0.020486,0.010297,1.000000,-0.332084,-0.024628,0.173488
country_Spain,0.004780,-0.001685,0.003868,-0.134892,0.009039,-0.013480,0.016732,-0.006482,-0.332084,1.000000,0.016889,-0.052667


In [8]:
corr_matrix[y.name].sort_values(ascending=False)

churn               1.000000
age                 0.285323
country_Germany     0.173488
balance             0.118533
estimated_salary    0.012097
credit_card        -0.007138
tenure             -0.014001
credit_score       -0.027094
products_number    -0.047820
country_Spain      -0.052667
gender_Male        -0.106512
active_member      -0.156128
Name: churn, dtype: float64

In [9]:
y.value_counts(normalize=True)*100

churn
0    79.63
1    20.37
Name: proportion, dtype: float64

# Hold-Out CV (2)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [11]:
cat_features = ['country', 'gender']

train_pool = Pool(data = X_train, label = y_train, cat_features= cat_features)
test_pool = Pool(data = X_test, label = y_test, cat_features= cat_features)

In [12]:
model = CatBoostClassifier(random_state=123, depth=2, eval_metric='AUC', verbose=0)

model.fit(train_pool, eval_set=test_pool, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [13]:
model.predict_proba(test_pool)

array([[0.92675708, 0.07324292],
       [0.96060165, 0.03939835],
       [0.86865975, 0.13134025],
       ...,
       [0.27077782, 0.72922218],
       [0.86695175, 0.13304825],
       [0.72574284, 0.27425716]])

In [14]:
y_pred_proba_test = model.predict_proba(test_pool)[:, 1]
auc_test = roc_auc_score(y_test, y_pred_proba_test)

In [15]:
y_pred_proba_train = model.predict_proba(train_pool)[:, 1]
auc_train = roc_auc_score(y_train, y_pred_proba_train)

In [16]:
auc_test, auc_train

(0.8762522741631317, 0.8785998707660174)

### 3 другие метрики (ps)

In [17]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

In [18]:
y_pred = model.predict(X_test)

In [19]:
hold_accuracy = accuracy_score(y_test, y_pred)
hold_precision = precision_score(y_test, y_pred)
hold_recall =recall_score(y_test, y_pred)
print(hold_accuracy)
print(hold_precision)
print(hold_recall)

0.8665
0.7441860465116279
0.48854961832061067


In [20]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1541   66]
 [ 201  192]]


### Таблица с метриками (7)

In [21]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

y_proba = model.predict_proba(X_test)[:, 1]
thresholds = [0.05, 0.1, 0.15, 0.25, 0.3]

table = []

for i in thresholds:
    y_pred_i = (y_proba >= i).astype(int)
    accuracy = accuracy_score(y_test, y_pred_i)
    precision = precision_score(y_test, y_pred_i)
    recall = recall_score(y_test, y_pred_i)
    f1 = f1_score(y_test, y_pred_i)
    
    table.append({
        'Threshold': i,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    })

metrics_df = pd.DataFrame(table)
metrics_df

,Threshold,Accuracy,Precision,Recall,F1 Score
0,0.05,0.4590,0.261592,0.961832,0.411317
1,0.10,0.6400,0.343840,0.916031,0.500000
2,0.15,0.7400,0.419722,0.844784,0.560811
3,0.25,0.8435,0.577519,0.758270,0.655666
4,0.30,0.8570,0.622426,0.692112,0.655422


# k-Fold CV (3)

In [22]:
from sklearn.model_selection import cross_val_score

model = CatBoostClassifier(random_state=123, depth=2, iterations=100, eval_metric='AUC', verbose=0, cat_features = cat_features)

kfold_scores = cross_val_score(model, X, y, cv = 5, scoring='roc_auc')

In [23]:
kfold_scores

array([0.86504366, 0.86521686, 0.86320758, 0.8648587 , 0.85962233])

In [24]:
kfold_mean = kfold_scores.mean()
kfold_std = kfold_scores.std()
print(kfold_mean)
print(kfold_std)

0.8635898275096494
0.0021098823028171854


# Stratified k-Fold CV (4)

Преимущество в том, что она лучше подходит когда у нас _дисбаланс_ классов (в нашем случае)

In [25]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits = 5)

skfold_scores = cross_val_score(model, X, y, cv = skf, scoring='roc_auc')

In [26]:
skfold_scores

array([0.86504366, 0.86521686, 0.86320758, 0.8648587 , 0.85962233])

In [27]:
skfold_mean = skfold_scores.mean()
skfold_std =  skfold_scores.std()
print(skfold_mean)
print(skfold_std)

0.8635898275096494
0.0021098823028171854


Сравнение с k-Fold

In [28]:
print(f'K-Fold: {kfold_scores}')
print(f'Stratify K-Flod: {skfold_scores}')

K-Fold: [0.86504366 0.86521686 0.86320758 0.8648587  0.85962233]
Stratify K-Flod: [0.86504366 0.86521686 0.86320758 0.8648587  0.85962233]


# Leave-One-Out Cross Validation – LOOCV (5)

In [29]:
X.shape

(10000, 10)

In [41]:
X_head = X.head(2000)
y_head = y.head(2000)

In [42]:
from sklearn.model_selection import LeaveOneOut

model = CatBoostClassifier(random_state=123, depth=2, iterations=100, eval_metric='AUC', verbose=0, cat_features = cat_features)

loo = LeaveOneOut()

loo_scores = cross_val_score(model, X_head, y_head, cv = loo, scoring='accuracy', n_jobs=-1)

это займет ~ 10мин 

если поменять X_hear, y_head на 1000 то будет ~ 5мин

In [45]:
loo_scores

array([0., 1., 1., ..., 1., 1., 1.])

In [46]:
loo_mean = loo_scores.mean()
loo_std =  loo_scores.std()
print(loo_mean)
print(loo_std)

0.855
0.352100837829165


### 3 другие метрики (ps)

In [47]:
model.fit(X_train, y_train)

In [48]:
y_pred = model.predict(X_test)

In [49]:
y_pred_proba = model.predict_proba(X_test)[:, 1] 

In [50]:
loo_accuracy = accuracy_score(y_test, y_pred)
loo_precision = precision_score(y_test, y_pred) 
loo_recall = recall_score(y_test, y_pred)
print(accuracy)
print(precision)
print(recall)

0.857
0.6224256292906178
0.6921119592875318


# Сравнительная таблица (6)

In [51]:
res = [
    {'type of CV': 'k-Fold CV', 'mean': kfold_mean, 'std': kfold_std},
    {'type of CV': 'Sk-Fold CV', 'mean': skfold_mean, 'std': skfold_std},
    {'type of CV': 'Hold-Out CV', 'mean': auc_test, 'std': '-'},
    {'type of CV': 'LOOCV', 'mean': loo_mean, 'std':loo_std}
]

In [52]:
results = pd.DataFrame(res)
results

,type of CV,mean,std
0,k-Fold CV,0.863590,0.00211
1,Sk-Fold CV,0.863590,0.00211
2,Hold-Out CV,0.876252,-
3,LOOCV,0.855000,0.352101
